All the results are compared in the presentation. Notebooks are just for the calculations.

In [1]:
import json
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn

from models import MyLSTM
from numpy.random import choice
from random import sample
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from utils import custom_split_train_test

In [2]:
district ='Kaithal'

In [3]:
os.listdir('data')    

['Dewas_NDVI.csv',
 'Dewas_points.csv',
 'Kaithal_NDVI.csv',
 'Kaithal_points.csv',
 'Karnal_NDVI.csv',
 'Karnal_points.csv']

In [4]:
data_points = pd.read_csv(f'data/{district}_points.csv')

In [5]:
data_ndvi = pd.read_csv(f'data/{district}_NDVI.csv')

In [6]:
data = data_points.merge(data_ndvi, left_on='gfid', right_on='gfid')

In [7]:
data.head()

,gfid,state,district,village,lon,lat,wheat,datenum,date,ndvi
0,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,0,2020-10-20,0.238
1,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,1,2020-10-21,0.235
2,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,2,2020-10-22,0.233
3,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,3,2020-10-23,0.231
4,52001,Haryana,Kaithal,Sirghar,76.420677,29.782459,1,4,2020-10-24,0.228


In [8]:
data_ndvi.head()

,gfid,datenum,date,ndvi
0,52001,0,2020-10-20,0.238
1,52001,1,2020-10-21,0.235
2,52001,2,2020-10-22,0.233
3,52001,3,2020-10-23,0.231
4,52001,4,2020-10-24,0.228


In [9]:
# Same inidicies as for CNN.
with open(f'{district}.json') as json_file:
    indices= json.load(json_file)
train = indices['train']
test = indices['test']

In [10]:
features = ['ndvi']
target = ['wheat']

In [11]:
# Take first 150 values for each time series as train data.
X_train = []
X_test = []
for id_ in train:
    d = data.loc[data['gfid']==id_, features].iloc[:150]
    X_train.append(d)
for id_ in test:
    d = data.loc[data['gfid']==id_, features].iloc[:150]
    X_test.append(d)
X_train = pd.concat(X_train)
X_test = pd.concat(X_test)

In [12]:
# Reshape data for LSTM.
h_1 = len(train)
h_2 = len(test)
w = 150
c = len(features)

X_train_tensors = Variable(torch.Tensor(X_train.values.reshape(h_1, w, c)))
X_test_tensors = Variable(torch.Tensor(X_test.values.reshape(h_2, w, c)))

y_train_tensors = Variable(torch.Tensor(data_points.loc[data_points['gfid'].isin(train), 'wheat'].values))
y_test_tensors = Variable(torch.Tensor(data_points.loc[data_points['gfid'].isin(test), 'wheat'].values)) 

y_train_tensors = y_train_tensors.type(torch.LongTensor)
y_test_tensors = y_test_tensors.type(torch.LongTensor)

In [14]:
input_size = len(features) #number of features
hidden_size = 32 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 2 #number of output classes 
batch_size = 10

In [15]:
lstm = MyLSTM(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1])

In [16]:
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(lstm1.parameters(), lr=0.001, momentum=0.9)

In [17]:
num_epochs = 100
for epoch in range(num_epochs):
    
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    outputs = lstm.forward(X_train_tensors) #forward pass

    # obtain the loss function
    loss = criterion(outputs, y_train_tensors)
    loss.backward() #calculates the loss of the loss function

    optimizer.step() #improve from loss, i.e backprop
    if epoch % 10 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

Epoch: 0, loss: 0.69265
Epoch: 10, loss: 0.69255
Epoch: 20, loss: 0.69233
Epoch: 30, loss: 0.69261
Epoch: 40, loss: 0.69246
Epoch: 50, loss: 0.69247
Epoch: 60, loss: 0.69242
Epoch: 70, loss: 0.69253
Epoch: 80, loss: 0.69267
Epoch: 90, loss: 0.69260


In [18]:
train_predict = lstm(X_train_tensors)

In [20]:
train_predict.argmax(dim=1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])

In [21]:
print(classification_report(y_train_tensors, torch.argmax(train_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        70
           1       0.53      1.00      0.70        80

    accuracy                           0.53       150
   macro avg       0.27      0.50      0.35       150
weighted avg       0.28      0.53      0.37       150



C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Also get results for train to detest over/under training.

In [22]:
test_predict = lstm.forward(X_test_tensors)

In [23]:
test_predict.argmax(dim=1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [24]:
print(classification_report(y_test_tensors, torch.argmax(test_predict, dim=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.57      1.00      0.73        24

    accuracy                           0.57        42
   macro avg       0.29      0.50      0.36        42
weighted avg       0.33      0.57      0.42        42



C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\60104252\Anaconda3\envs\ds\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
roc_auc_score(y_test_tensors, torch.argmax(test_predict, dim=1))

0.5

All the results are compared in the presentation. Notebooks are just for the calculations.